In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

## 3950 Assignment 1: Part 2

For this assignment we want to use some sort of tree based model to classify the data below. We have a very small training set, so overfitting is a very real concern. 

Some specifics for this assignment:
<ul>
<li>Please use the show_eda to control if EDA stuff is shown. I don't really need to see all the EDA stuff (nor do you after you've done it), so we can make it configurable with a variable to speed up time. Please set this FALSE when you submit, so I can run all and see the outcome without histograms etc...
<li>Please ensure that whatever model you end up with is in a variable named best at the end.
<li>Please use some pipeline in prepping the data. The test data is in an identical format to the training data, so whatever pipeline you've created for your training will work for the testing. 
<li>The accuracy scoring will be an average of accuracy and roc_auc. 
</ul>

### Grading Metrics
<ul>
<li><b>Pipeline Used - 10pts</b> The data loading needs to be in a pipeline. See the test part for illustration. When testing I'll call your pipe with the new data (format is identical to training), so any prep stuff should be in the pipeline. 
<li><b>Tree Based Model Used - 5pts</b> The model used for classification needs to be some variety of tree, beyond that it is up to you. 
<li><b>Accuracy - 5pts</b> The final accuracy acheived. This will be a rough ranking, I'm assuming most people will get a similar level of accuracy, marks will only be deducted if yours is far wosrse, as that's an indication that you probably didn't take any/many steps to improve things. 
<li><b>Clarity and Formatting - 5pts</b> Is it organized and can I read it?
    <ul>
    <li> <b>Note:</b> for this assignment, and in general, please get rid of my comments and replace them with your own. I'm going to read this, so all of these instructions aren't really required. Think of this as a template, get rid of the stuff that isn't needed, and leave only the things you need to explain your code. 
    </ul>
</ul>

For submission, please drop the URL for your repository in the dropbox.

In [4]:
name = "Esha Basharat"

#Control EDA. 
show_eda = False

In [5]:
#Load data
df = pd.read_csv("training.csv")
df = df.drop(columns={"id"})
df.sample(5)

,target,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200
197,0,0.748,0.018,0.670,0.265,0.011,0.616,0.961,0.384,0.930,...,0.601,0.291,0.219,0.880,0.252,0.016,0.271,0.301,0.512,0.914
89,1,0.981,0.992,0.725,0.163,0.124,0.856,0.501,0.983,0.471,...,0.211,0.371,0.257,0.941,0.545,0.700,0.525,0.620,0.314,0.229
109,1,0.204,0.559,0.314,0.523,0.662,0.385,0.486,0.311,0.636,...,0.693,0.896,0.768,0.143,0.731,0.769,0.437,0.526,0.669,0.946
222,1,0.056,0.866,0.309,0.296,0.576,0.451,0.365,0.938,0.465,...,0.367,0.121,0.847,0.144,0.518,0.539,0.232,0.323,0.230,0.904
177,1,0.072,0.308,0.007,0.626,0.809,0.578,0.003,0.472,0.014,...,0.944,0.792,0.130,0.911,0.867,0.043,0.570,0.261,0.059,0.989


In [6]:
#Display coloumn names 
print(df.columns)


Index(['target', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7',
       'var_8', 'var_9',
       ...
       'var_191', 'var_192', 'var_193', 'var_194', 'var_195', 'var_196',
       'var_197', 'var_198', 'var_199', 'var_200'],
      dtype='object', length=201)


In [7]:
#Load the dataset 
test_df = pd.read_csv("training.csv")  
test_df = test_df.drop(columns=["id"]) #drop the id coloumn 

# Target coloumn is dropped from the df 
test_X = test_df.drop(columns=["target"])
test_y = np.array(test_df["target"]).reshape(-1, 1)


In [20]:
#EDA 
def show_eda(data):
    # Display's the basic information about the dataset
    print(data.info())

    # Visualize the distribution of the target variable
    sns.countplot(x='target', data=data)
    plt.title('Distribution of Target Variable')
    plt.show()

    # Select a few key numerical features for distribution plots
    numeric_features = ['var_1', 'var_2', 'var_3']  # using the first 3 variable coloumns 
    for feature in numeric_features:
        sns.histplot(data[feature], kde=True)
        plt.title(f'Distribution of {feature}')
        plt.show()

    # correlation matrix for numerical features
    corr_matrix = data[numeric_features].corr()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
    plt.title('Correlation Matrix for Numerical Features')
    plt.show()

# Set the EDA to false 
show_eda_flag = False


In [9]:
#Craete a pipeline 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

# Define features and target variable
X = df.drop(columns=["target"])
y = df["target"]

# Create a pipeline for data preparation and modeling
numeric_features = X.select_dtypes(include=[np.number]).columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        # Add more transformers for categorical features if needed
    ])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])


In [10]:
#Train the model 
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7', 'var_8',
       'var_9', 'var_10',
       ...
       'var_191', 'var_192', 'var_193', 'var_194', 'var_195', 'var_196',
       'var_197', 'var_198', 'var_199', 'var_200'],
      dtype='object', length=200))])),
                ('classifier', RandomForestClassifier())])

In [17]:
#Evaluate accuracy 
from sklearn.metrics import accuracy_score

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Print the result
print(f"Accuracy: {accuracy}")


Accuracy: 0.56


In [19]:
#Evaluate accuracy use hyperparameters to make it better 
from sklearn.model_selection import GridSearchCV
# Define hyperparameter grid
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20],
    # Add more hyperparameters to tune
}

# Create GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')

# Fit the model with the best hyperparameters
grid_search.fit(X_train, y_train)

# Get the best model
best = grid_search.best_estimator_

# Print the accuracy on the test set using the best model
print("Accuracy on Test Set:", best.score(X_test, y_test))





Accuracy on Test Set: 0.72


### Starting

For this assignment, you have a small training set, so combatting overfitting is key in being accurate!

In [ ]:
df.shape

(250, 201)

#### Do Modelling Stuff

Make a tree model (of some vareity) and make it fit well. Keep in mind the possibility of your tree overfitting, and think of steps you may need to combat that shoudl it occur. 

### Finishing

At the conclusion, please name your best model "best". If you look down below in the testing stuff, it should be usable to score as "best". 

You should be able to call it like this and it should work (with whatever data names you have)

### Testing

Please leave the stuff below as-is in your file. 

This will take your best model and score it with the test data. If you want to test to make sure that yours works, make a copy of the data file and rename it testing.csv, then make sure this runs ok. I will do the same, but the contents of my test file will be different. 

In [ ]:
#Load Test Data
test_df = pd.read_csv("testing.csv")
test_df = test_df.drop(columns={"id"})
#Create tests and score
test_y = np.array(test_df["target"]).reshape(-1,1)
test_X = np.array(test_df.drop(columns=["target"]))

preds = best.predict(test_X)

roc_score = roc_auc_score(test_y, preds)
acc_score = accuracy_score(test_y, preds)

print(roc_score)
print(acc_score)
print(name, np.mean([roc_score, acc_score]))

### What Accuracy Changes Were Used

Please list here what you did to try to increase accuracy and/or limit overfitting:
<ul> Pipeline implementation: I implemented a pipeline that includes data preprocessing (scaling) and a tree based model (RandomForestClassifier).     This helps maintain training and testing phases 
<li> Hyperparameter Tuning: I used GridSearchCV which is used to perform hyperparameter tuning for the RandomForestClassifier. This allows searching over a predefined grid of hyperparamter values to find combinations that yield best performance. 
<li> Model training and evaluation: I split the data into training and testing sets and trained the model using the training set. 